<a href="https://colab.research.google.com/github/humamfa/ImageClassification_Animals/blob/main/Image_Classification_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
import tensorflow as tf
!pip install opendatasets
import opendatasets as od
import nltk
nltk.download('wordnet')
import pickle
!pip install split-folders
import splitfolders
import zipfile,os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow import keras
from tensorflow.keras import layers
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional, Reshape, SpatialDropout1D
from keras.datasets import imdb

[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
od.download('https://www.kaggle.com/datasets/antobenedetti/animals')

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: humamfa
Your Kaggle Key: ··········


100%|██████████| 882M/882M [00:45<00:00, 20.5MB/s]


In [ ]:
base_dir = '/content/animals/animals/train'
train_dir = '/content/output/train'
validation_dir = '/content/output/val'

In [ ]:
%pip install split-folders
import splitfolders

splitfolders.ratio(base_dir, output="output", seed=6666, ratio=(.8, .2), group_prefix=None)

Copying files: 13474 files [00:07, 1845.41 files/s]


In [ ]:
train_datagen = ImageDataGenerator(
                    rescale=1./255,
                    rotation_range=20,
                    horizontal_flip=True,
                    shear_range = 0.2,
                    validation_split = 0.2,
                    fill_mode = 'wrap',
                    )

validation_datagen = ImageDataGenerator(
                    rescale=1./255,
                    rotation_range=20,
                    horizontal_flip=True,
                    shear_range = 0.2,
                    fill_mode = 'nearest',)

In [ ]:
train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(100, 150),
        batch_size=64,
        class_mode='categorical')

validation_generator = validation_datagen.flow_from_directory(
        validation_dir,
        target_size=(100, 150),
        batch_size=64,
        class_mode='categorical')

Found 10778 images belonging to 5 classes.
Found 2696 images belonging to 5 classes.


In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(100, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(256, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax')
])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 98, 148, 32)       896       
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 49, 74, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_5 (Conv2D)           (None, 47, 72, 64)        18496     
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 23, 36, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_6 (Conv2D)           (None, 21, 34, 128)       73856     
                                                                 
 max_pooling2d_6 (MaxPoolin  (None, 10, 17, 128)      

In [ ]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

opt = tf.keras.optimizers.RMSprop(learning_rate=0.0001)

model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy')>0.92 and logs.get('val_accuracy')>0.92):
      print("\nAkurasi telah mencapai >92%!")
      self.model.stop_training = True
callbacks = myCallback()

history = model.fit(
      train_generator,
      steps_per_epoch=35,
      epochs=1000,
      validation_data=validation_generator,
      validation_steps=15,
      verbose=2,
      callbacks=[callbacks])

Epoch 1/1000
35/35 - 25s - loss: 1.6011 - accuracy: 0.2415 - val_loss: 1.5780 - val_accuracy: 0.2396 - 25s/epoch - 718ms/step
Epoch 2/1000
35/35 - 20s - loss: 1.5348 - accuracy: 0.3442 - val_loss: 1.4980 - val_accuracy: 0.3292 - 20s/epoch - 579ms/step
Epoch 3/1000
35/35 - 24s - loss: 1.4450 - accuracy: 0.4083 - val_loss: 1.4173 - val_accuracy: 0.4156 - 24s/epoch - 677ms/step
Epoch 4/1000
35/35 - 22s - loss: 1.3939 - accuracy: 0.4214 - val_loss: 1.4033 - val_accuracy: 0.4198 - 22s/epoch - 623ms/step
Epoch 5/1000
35/35 - 23s - loss: 1.3481 - accuracy: 0.4537 - val_loss: 1.3363 - val_accuracy: 0.4552 - 23s/epoch - 667ms/step
Epoch 6/1000
35/35 - 19s - loss: 1.2973 - accuracy: 0.4826 - val_loss: 1.3752 - val_accuracy: 0.4344 - 19s/epoch - 543ms/step
Epoch 7/1000
35/35 - 21s - loss: 1.2810 - accuracy: 0.4951 - val_loss: 1.2234 - val_accuracy: 0.5208 - 21s/epoch - 613ms/step
Epoch 8/1000
35/35 - 20s - loss: 1.2483 - accuracy: 0.5059 - val_loss: 1.2538 - val_accuracy: 0.5021 - 20s/epoch - 559

In [ ]:
import pathlib

export_dir = 'saved_model/'
tf.saved_model.save(model, export_dir)

converter = tf.lite.TFLiteConverter.from_saved_model(export_dir)
tflite_model = converter.convert()

tflite_model_file = pathlib.Path('animals.tflite')
tflite_model_file.write_bytes(tflite_model)

8904736

In [ ]:
plt.plot(history.history['loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train'], loc='upper right')
plt.show()

In [ ]:
plt.plot(history.history['accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train'], loc='upper right')
plt.show()